In [43]:
import pandas as pd

import random
import re
import pickle

In [2]:
dataset = pd.read_csv("reviews.csv")
dataset.head(3)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,207.156.210.106
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,193.93.167.87
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,218.58.147.115


In [3]:
### WORKING ON TWITTER DATA FOR SENTIMENTAL ANALYSIS

with open("Pickle Files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("Pickle Files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)

def getSentiment(text):

    # PREPROCESSING THE DATASET
    text = str(text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]
            

## 1. different sentiment in review headline and review body

In [34]:
blocked_reviews = []
for i in range(len(dataset)):
        if( getSentiment( dataset["review_headline"][i] ) != getSentiment( dataset["review_body"][i] ) ):
            blocked_reviews.append(dataset["review_id"][i])

## 2. Users which are posting either all positive or negative reviews on different products of same brand


In [44]:
dataset = pd.read_csv("sentiment-for-a-company-product-by-same-customer.csv")

In [46]:
# users which are posting either all positive or negative reviews on different products of same brand

customers = dataset.groupby("customer_id")
customer_list = dataset["customer_id"].unique()

blocked_users = []
size = len(customer_list.tolist())
index = 0
for i in range(size):
    brand_df = customers.get_group(customer_list[i])    
    brands = brand_df.groupby("product_parent")
    brands_list = brand_df["product_parent"].unique()
    
    no_of_brands = len(brands_list.tolist())
    for j in range(no_of_brands):
        
        product_df = brands.get_group(brands_list[j])
        no_of_products = len(product_df["product_id"])
        
        sentiment = getSentiment(product_df["review_body"][index])
                
        isSameSentiment = True
        if no_of_products==2:
            break
        for k in range(no_of_products):
            text = str(product_df["review_body"][index+k])
            if getSentiment(text)!=sentiment :
                isSameSentiment = False
                break;
        if(isSameSentiment):
            blocked_users.append(customer_list[i])
            break
    if(isSameSentiment):
        break;
    index += k

In [47]:
blocked_users

[20422322]

In [ ]:
'''
dataset.set_index("customer_id",inplace=True)
for customer in blocked_users:
    dataset.drop(customer,inplace=True)
dataset.reset_index()
'''

## 3.Reviews from same IP either all positive or negative reviews on different products of same brand

In [48]:
dataset = pd.read_csv("sentiment-for-a-company-product-by-same-IP.csv")

In [50]:
ip = dataset.groupby("IP Address")
ip_list = dataset["IP Address"].unique()

# Reviews from same IP either all positive or negative reviews on different products of same brand

blocked_users = []
size = len(ip_list.tolist())
index = 0
for i in range(size):
    brand_df = ip.get_group(ip_list[i])
    
    brands = brand_df.groupby("product_parent")
    brands_list = brand_df["product_parent"].unique()
    
    no_of_brands = len(brands_list.tolist())
    for j in range(no_of_brands):
        
        product_df = brands.get_group(brands_list[j])
        no_of_products = len(product_df["product_id"])
        
        sentiment = getSentiment(product_df["review_body"][index])
                
        isSameSentiment = True
        if no_of_products<=2:
            break
        for k in range(no_of_products):
            text = str(product_df["review_body"][index+k])
            if getSentiment(text)!=sentiment :
                isSameSentiment = False
                break;
        if(isSameSentiment):
            blocked_users.append(ip_list[i])
            break
    if(isSameSentiment):
        break;
    index +=k

In [51]:
blocked_users

['193.93.167.87']

In [53]:
'''
dataset.set_index("IP Address",inplace=True)
for ip in blocked_users:
    dataset.drop(ip,inplace=True)
dataset.reset_index()
'''

'\ndataset.set_index("IP Address",inplace=True)\nfor ip in blocked_users:\n    dataset.drop(ip,inplace=True)\ndataset.reset_index()\n'

## 4. User posting (>3) reviews on the same day with all the reviews are either positive or negative.

In [5]:
dataset.sort_values("customer_id",inplace=True)

In [8]:
dataset.head(1)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
24496,US,10071,R1XALU24Y932WC,B0048DLA90,864204912,NEW Click Wheel Flex Cable For Apple iPod Nano...,Mobile_Electronics,5,0.0,0.0,N,Y,Five Stars,Just what I needed. Thanks.,2014-07-31,217.163.92.95


In [4]:
dataset = pd.read_csv("same-day-multiple-reviews with same sentiment.csv")

In [29]:
customer_group = dataset.groupby("customer_id")
customer_group_list = dataset["customer_id"].unique().tolist()
index=0

for i in range(len(customer_group_list)):
    customer_reviews = customer_group.get_group( customer_group_list[i] )
    
    dates_list = customer_reviews["review_date"].unique().tolist()
    print(dates_list)
    reviews_by_date = customer_reviews.groupby("review_date");
    
    for j in range(len(dates_list)):
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        
        for k in range(len(df)):
            text = df["review_body"][index+k]
            if(getSentiment(text) == 0):
                reviews_by_date_for_neg.append(df["review_id"][index+k])
            else:
                reviews_by_date_for_pos.append(df["review_id"][index+k])
        print(len(reviews_by_date_for_pos) , reviews_by_date_for_pos)
        print(len(reviews_by_date_for_neg) , reviews_by_date_for_neg)
        # count, review id of reviews with same sentiment
        index += k+1
        

['2015-08-31']
3 ['R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P']
1 ['RRB9C05HDOD4O']
['2015-09-31']
4 ['R26I2RI1GFV8QG', 'RY8DDL22YG4R5', 'R2AT2426ZHFUHH', 'R3RRXU2R23NMQ9']
0 []


dict_keys(['2015-09-31'])

## 5. Reviews(>3) from same IP on the same day with all the reviews are either positive or negative.

In [41]:
dataset = pd.read_csv("same-day-multiple-reviews with same sentiment from same IP.csv")

In [42]:
ip_group = dataset.groupby("IP Address")
ip_list = dataset["IP Address"].unique().tolist()
index=0

for i in range(len(ip_list)):
    reviews = ip_group.get_group( ip_list[i] )
    
    dates_list = reviews["review_date"].unique().tolist()
    print(dates_list)
    reviews_by_date = reviews.groupby("review_date");
    
    for j in range(len(dates_list)):
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        
        for k in range(len(df)):
            text = df["review_body"][index+k]
            if(getSentiment(text) == 0):
                reviews_by_date_for_neg.append(df["review_id"][index+k])
            else:
                reviews_by_date_for_pos.append(df["review_id"][index+k])
        print(len(reviews_by_date_for_pos) , reviews_by_date_for_pos)
        print(len(reviews_by_date_for_neg) , reviews_by_date_for_neg)
        # count, review id of reviews with same sentiment
        index += k+1
        

['2015-08-31']
3 ['R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P']
1 ['RRB9C05HDOD4O']
['2015-09-31']
4 ['R26I2RI1GFV8QG', 'RY8DDL22YG4R5', 'R2AT2426ZHFUHH', 'R3RRXU2R23NMQ9']
0 []
